In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as python_random

import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import regularizers
import pickle

2023-03-05 20:21:16.735450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
aggregated_data = pickle.load( open( '../data/processed/20230304/aggregated_data', 'rb' ) )

In [3]:
[train_mps_raw_pca, valid_mps_raw_pca, test_mps_raw_pca,
    train_mps_bg_pca, valid_mps_bg_pca, test_mps_bg_pca,
    train_mps_fg_pca, valid_mps_fg_pca, test_mps_fg_pca,
    train_indices_raw_pca, valid_indices_raw_pca, test_indices_raw_pca,
    train_indices_bg_pca, valid_indices_bg_pca, test_indices_bg_pca,
    train_indices_fg_pca, valid_indices_fg_pca, test_indices_fg_pca,
    train_embedding_raw_pca, valid_embedding_raw_pca, test_embedding_raw_pca,
    train_embedding_bg_pca, valid_embedding_bg_pca, test_embedding_bg_pca,
    train_embedding_fg_pca, valid_embedding_fg_pca, test_embedding_fg_pca,
    y_train, y_valid, y_test] = aggregated_data

In [4]:
train_data_raw = np.concatenate((train_mps_raw_pca, train_indices_raw_pca, train_embedding_raw_pca), axis = 1)
valid_data_raw = np.concatenate((valid_mps_raw_pca, valid_indices_raw_pca, valid_embedding_raw_pca), axis = 1)
test_data_raw = np.concatenate((test_mps_raw_pca, test_indices_raw_pca, test_embedding_raw_pca), axis = 1)

train_data_bg = np.concatenate((train_mps_bg_pca, train_indices_bg_pca, train_embedding_bg_pca), axis = 1)
valid_data_bg = np.concatenate((valid_mps_bg_pca, valid_indices_bg_pca, valid_embedding_bg_pca), axis = 1)
test_data_bg = np.concatenate((test_mps_bg_pca, test_indices_bg_pca, test_embedding_bg_pca), axis = 1)

train_data_fg = np.concatenate((train_mps_fg_pca, train_indices_fg_pca, train_embedding_fg_pca), axis = 1)
valid_data_fg = np.concatenate((valid_mps_fg_pca, valid_indices_fg_pca, valid_embedding_fg_pca), axis = 1)
test_data_fg = np.concatenate((test_mps_fg_pca, test_indices_fg_pca, test_embedding_fg_pca), axis = 1)

train_data_all = np.concatenate((train_data_raw, train_data_bg, train_data_fg), axis = 1)
valid_data_all = np.concatenate((valid_data_raw, valid_data_bg, valid_data_fg), axis = 1)
test_data_all = np.concatenate((test_data_raw, test_data_bg, test_data_fg), axis = 1)

# Tensorflow parameter tuning

In [23]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt


In [40]:
def build_model_430(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(430), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.3, max_value=0.7, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=25, max_value=400, step=25),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model



In [41]:
def build_model_1290(hp):

    model = keras.Sequential()
    model.add(layers.Input(shape=(1290), dtype=tf.float32))
    drop_rate = hp.Float("dr", min_value=0.3, max_value=0.7, sampling="linear")
#         if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=drop_rate))
    # Tune the number of layers.
    for i in range(hp.Int("num_layers", 1, 10)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=25, max_value=400, step=25),
                activation=hp.Choice("activation", ["relu", "tanh"]),
            )
        )
    model.add(layers.Dense(2))
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )

    return model

In [44]:
tuner_raw = kt.BayesianOptimization(build_model_430,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_raw_0305_v2')

tuner_bg = kt.BayesianOptimization(build_model_430,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_bg_0305_v2')

tuner_fg = kt.BayesianOptimization(build_model_430,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_fg_0305_v2')

tuner_all = kt.BayesianOptimization(build_model_1290,
                     objective='val_accuracy',
                     max_trials=100,
                     num_initial_points=25,
                     seed=23,
#                      overwrite=True,
                     directory='../model',
#                      distribution_strategy=tf.distribute.MirroredStrategy(),
                     project_name='kt_Bayesian_all_0305_v2')

In [45]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [47]:
tuner_raw.search(train_data_raw, y_train, epochs=50, validation_data=(valid_data_raw, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 62 Complete [00h 00m 28s]
val_accuracy: 0.6894701719284058

Best val_accuracy So Far: 0.7082495093345642
Total elapsed time: 00h 21m 32s

Search: Running Trial #63

Value             |Best Value So Far |Hyperparameter
0.7               |0.34518           |dr
3                 |6                 |num_layers
250               |225               |units_0
tanh              |relu              |activation
0.0001            |0.00014284        |lr
25                |250               |units_1
25                |225               |units_2
25                |225               |units_3
25                |150               |units_4
25                |275               |units_5
25                |275               |units_6
25                |100               |units_7
400               |None              |units_8
25                |None              |units_9

Epoch 1/50
373/373 [==============================] - 2s 4ms/step - loss: 0.7186 - accuracy: 0.5380 - val_loss: 0.6390 - val_accuracy: 

KeyboardInterrupt: 

In [ ]:
tuner_bg.search(train_data_bg, y_train, epochs=50, validation_data=(valid_data_bg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [ ]:
tuner_fg.search(train_data_fg, y_train, epochs=50, validation_data=(valid_data_fg, y_valid), callbacks=[stop_early], use_multiprocessing=True)

In [46]:
tuner_all.search(train_data_all, y_train, epochs=50, validation_data=(valid_data_all, y_valid), callbacks=[stop_early], use_multiprocessing=True)

Trial 100 Complete [00h 01m 32s]
val_accuracy: 0.7022132873535156

Best val_accuracy So Far: 0.7169684767723083
Total elapsed time: 01h 20m 03s
INFO:tensorflow:Oracle triggered exit


In [21]:
tuner_raw.results_summary(1)

Results summary
Results in ../model/kt_Bayesian_raw_0305
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.5
dropout: True
num_layers: 10
units_0: 260
activation: relu
lr: 0.0001
units_1: 400
units_2: 20
units_3: 400
units_4: 380
units_5: 180
units_6: 400
units_7: 20
units_8: 260
units_9: 20
Score: 0.7162978053092957


In [22]:
tuner_bg.results_summary(1)

Results summary
Results in ../model/kt_Bayesian_bg_0305
Showing 1 best trials
Trial summary
Hyperparameters:
dr: 0.5
dropout: True
num_layers: 1
units_0: 320
activation: tanh
lr: 0.0001
units_1: 380
units_2: 400
units_3: 20
units_4: 180
units_5: 400
units_6: 400
units_7: 20
units_8: 20
units_9: 400
Score: 0.7116029262542725
